<a href="https://colab.research.google.com/github/hwangsaeyeon/LG-Aimers/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/aimers/

!unzip -qq "/content/drive/MyDrive/aimers/open.zip"

/content/drive/MyDrive/aimers


In [53]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/drive/MyDrive/aimers/train.csv",)
test = pd.read_csv("/content/drive/MyDrive/aimers/test.csv",)
submission = pd.read_csv("/content/drive/MyDrive/aimers/sample_submission.csv",)

In [4]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

In [33]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

#Data

In [54]:
#결측치 0으로 채움
train = train.fillna(0)
test = test.fillna(0)

In [55]:
train_x = train.filter(regex='X') # Input : X Feature
train_y = train.filter(regex='Y') # Output : Y Feature

#Regression Model

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3)

In [57]:
LR = MultiOutputRegressor(LinearRegression()).fit(X_train,y_train)
print('Done.')

Done.


In [66]:
from sklearn import metrics
#normalized rmse 
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [67]:
y_pred = LR.predict(X_test)
print('score : ',lg_nrmse(np.array(y_test),y_pred))

score :  1.9812028097417218


#Inference

In [68]:
test_x = test.drop(columns=['ID'])

In [69]:
preds = LR.predict(test_x)
print('Done.')

Done.


In [70]:
preds.shape

(39608, 14)

#Submit

In [ ]:
for idx, col in enumerate(submission.columns):
    if col=='ID':
        continue
    submission[col] = preds[:,idx-1]
print('Done.')

In [ ]:
submission.to_csv('./submit.csv', index=False)